# Molecular Dynamics

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import Stationary, ZeroRotation, MaxwellBoltzmannDistribution
from xtb.ase.calculator import XTB
from mace.calculators import MACECalculator
from aseMolec import anaAtoms as aa
import subprocess

/usr/local/lib/python3.10/dist-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.d

In [17]:
output_dir = "moldyn"
os.makedirs(output_dir, exist_ok=True)

def run_md(name, init_conf, temp, calc, steps, interval):
    """Run MD with Langevin thermostat and save trajectory + return log data."""
    conf = init_conf.copy()
    conf.calc = calc

    # Initialize velocities
    MaxwellBoltzmannDistribution(conf, temperature_K=300)
    Stationary(conf)
    ZeroRotation(conf)

    traj_file = os.path.join(output_dir, f"{name}.xyz")
    if os.path.exists(traj_file):
        os.remove(traj_file)

    time_fs, temperatures, energies = [], [], []
    dyn = Langevin(conf, 1.0 * units.fs, temperature_K=temp, friction=0.1)

    def log():
        dyn.atoms.write(traj_file, append=True)
        time_fs.append(dyn.get_time() / units.fs)
        temperatures.append(dyn.atoms.get_temperature())
        energies.append(dyn.atoms.get_potential_energy() / len(dyn.atoms))

    dyn.attach(log, interval=interval)
    dyn.run(steps)

    return np.array(time_fs), np.array(temperatures), np.array(energies)

def plot_temperature(t1, temp1, t2, temp2, label1, label2, temp_target, N, out_file, ylim=None):
    """Plot temperature vs time with equilibrium fluctuation line."""
    fluctuation = temp_target * np.sqrt(2 / (3 * N))

    plt.figure(figsize=(6, 4))
    plt.plot(t1, temp1, label=f'{label1} T(t)')
    plt.plot(t2, temp2, label=f'{label2} T(t)')

    # Plot mean temperature lines
    plt.axhline(np.mean(temp1), linestyle='--', color='blue', label=f'{label1} ⟨T⟩ = {np.mean(temp1):.1f} K')
    plt.axhline(np.mean(temp2), linestyle='--', color='red', label=f'{label2} ⟨T⟩ = {np.mean(temp2):.1f} K')

    # Plot target temperature (solid gray)
    plt.axhline(temp_target, linestyle='-', color='gray', label=f'{temp_target} K')

    # Plot theoretical fluctuation bounds (dotted gray)
    plt.axhline(temp_target + fluctuation, linestyle=':', color='gray', label=f'{temp_target} ± {fluctuation:.1f} K')
    plt.axhline(temp_target - fluctuation, linestyle=':', color='gray')
    
    plt.xlabel("Time (fs)")
    plt.ylabel("Temperature (K)")
    plt.title("Temperature Over Time")
    plt.legend()

    if ylim:
        plt.ylim(*ylim)  # (0, 700)
    
    plt.tight_layout()
    plt.savefig(out_file, dpi=300)
    plt.close()

def plot_energy(t1, E1, t2, E2, label1, label2, out_file):
    """Plot energy per atom over time."""
    plt.figure(figsize=(6, 4))
    plt.plot(t1, E1, label=label1)
    plt.plot(t2, E2, label=label2)
    plt.xlabel("Time (fs)")
    plt.ylabel("Energy (eV/atom)")
    plt.title("Potential Energy per Atom")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_file, dpi=300)
    plt.close()

def plot_rdfs(file1, file2, tags, out_prefix, label1, label2):
    traj1 = read(file1, "50:")
    traj2 = read(file2, "50:")

    for traj in [traj1, traj2]:
        for at in traj:
            at.pbc = True
            at.cell = [100, 100, 100]

    for tag in tags:
        rdf1 = aa.compute_rdfs_traj_avg(traj1, rmax=5, nbins=50)
        rdf2 = aa.compute_rdfs_traj_avg(traj2, rmax=5, nbins=50)

        plt.figure(figsize=(5, 3.5))
        plt.rcParams.update({'font.size': 15})
        plt.plot(rdf1[1], rdf1[0][tag], label=label1)
        plt.plot(rdf2[1], rdf2[0][tag], label=label2)
        plt.xlabel(r"R ($\rm \AA$)")
        plt.ylabel(f"RDF {tag}")
        plt.legend()
        plt.title(f"RDF {tag}")
        plt.tight_layout()
        plt.savefig(f"{out_prefix}_rdf_{tag}.png", dpi=300)
        plt.close()

def load_msd(file):
    t, msd = np.loadtxt(file, comments="#", usecols=(0, 2), unpack=True)
    return t * 1000, msd  # convert ps → fs


def compute_diffusion(t_fs, msd, t_max_fs=100):
    """Compute diffusion constant by fitting MSD vs time with y = a*x (through origin)."""
    mask = t_fs <= t_max_fs
    t_fit = t_fs[mask] / 1000  # fs → ps
    msd_fit = msd[mask]

    # Linear fit through origin: slope = sum(x*y)/sum(x²)
    numerator = np.sum(t_fit * msd_fit)
    denominator = np.sum(t_fit ** 2)
    slope = numerator / denominator
    D = slope / 6  # Å²/ps

    return D, slope

In [3]:
# === MD for single molecule ===
mol_conf = [a for a in read('data/solvent_molecs.xyz', ':') if a.info.get('Nmols') == 1][0]
N_mol = len(mol_conf)
T_target_mol = 1200

mace_calc = MACECalculator(model_paths=['MACE_models/mace_learncurve_train4000_swa_compiled.model'],
                            device='cuda', default_dtype='float64')
xtb_calc = XTB(method="GFN2-xTB")

mol_mace_t, mol_mace_temp, mol_mace_E = run_md("mace_md_molecule", mol_conf, temp=T_target_mol, calc=mace_calc, steps=5000, interval=10)
mol_xtb_t, mol_xtb_temp, mol_xtb_E = run_md("xtb_md_molecule", mol_conf, temp=T_target_mol, calc=xtb_calc, steps=5000, interval=10)

np.savez(os.path.join(output_dir, "mol_mace_4000.npz"), t=mol_mace_t, T=mol_mace_temp, E=mol_mace_E, N=N_mol, T_target=T_target_mol)
np.savez(os.path.join(output_dir, "mol_xtb.npz"), t=mol_xtb_t, T=mol_xtb_temp, E=mol_xtb_E, N=N_mol, T_target=T_target_mol)

/usr/local/lib/python3.10/dist-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)
/usr/local/

In [4]:
# === Call MSD.py for both single molecule MDs ===
subprocess.run(["python3", "MSD.py", os.path.join(output_dir, "mace_md_molecule.xyz"),
                "--out", os.path.join(output_dir, "msd_mace_md_molecule.dat"),
                "--dt", "1", "--png", os.path.join(output_dir, "msd_mace_md_molecule.png"), "--skip", "1"])

subprocess.run(["python3", "MSD.py", os.path.join(output_dir, "xtb_md_molecule.xyz"),
                "--out", os.path.join(output_dir, "msd_xtb_md_molecule.dat"),
                "--dt", "1", "--png", os.path.join(output_dir, "msd_xtb_md_molecule.png"), "--skip", "1"])

✔  wrote moldyn/msd_mace_md_molecule.dat  with 501 lines
✔  wrote moldyn/msd_mace_md_molecule.png
✔  wrote moldyn/msd_xtb_md_molecule.dat  with 501 lines
✔  wrote moldyn/msd_xtb_md_molecule.png


CompletedProcess(args=['python3', 'MSD.py', 'moldyn/xtb_md_molecule.xyz', '--out', 'moldyn/msd_xtb_md_molecule.dat', '--dt', '1', '--png', 'moldyn/msd_xtb_md_molecule.png', '--skip', '1'], returncode=0)

In [5]:
# === MD for liquid ===
# Needs fine tuned model
liq_conf = read("data/input2.xyz")
liq_conf.center()
N_liq = len(liq_conf)
T_target_liq = 500

mace_liq_calc = MACECalculator(model_paths=['MACE_models/mace_learncurve_train4000_swa_compiled.model'],
                                device='cuda', default_dtype='float64')
finetuned_calc = MACECalculator(model_paths=['finetuned_MACE_compiled.model'],
                                device='cuda', default_dtype='float64')

liq_mace_t, liq_mace_temp, liq_mace_E = run_md("mace_md_input2", liq_conf, temp=T_target_liq, calc=mace_liq_calc, steps=5000, interval=10)
liq_ft_t, liq_ft_temp, liq_ft_E = run_md("mace_finetuned_md", liq_conf, temp=T_target_liq, calc=finetuned_calc, steps=5000, interval=10)

np.savez(os.path.join(output_dir, "liq_mace_4000.npz"), t=liq_mace_t, T=liq_mace_temp, E=liq_mace_E, N=N_liq, T_target=T_target_liq)
np.savez(os.path.join(output_dir, "liq_finetuned.npz"), t=liq_ft_t, T=liq_ft_temp, E=liq_ft_E, N=N_liq, T_target=T_target_liq)

/usr/local/lib/python3.10/dist-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)
/usr/local/

In [6]:
# === Call MSD.py for both liquid MDs ===
subprocess.run(["python3", "MSD.py", os.path.join(output_dir, "mace_md_input2.xyz"),
                "--out", os.path.join(output_dir, "msd_mace_input2.dat"),
                "--dt", "1", "--png", os.path.join(output_dir, "msd_mace_input2.png"), "--skip", "1"])

subprocess.run(["python3", "MSD.py", os.path.join(output_dir, "mace_finetuned_md.xyz"),
                "--out", os.path.join(output_dir, "msd_mace_finetuned.dat"),
                "--dt", "1", "--png", os.path.join(output_dir, "msd_mace_finetuned.png"), "--skip", "1"])

✔  wrote moldyn/msd_mace_input2.dat  with 501 lines
✔  wrote moldyn/msd_mace_input2.png
✔  wrote moldyn/msd_mace_finetuned.dat  with 501 lines
✔  wrote moldyn/msd_mace_finetuned.png


CompletedProcess(args=['python3', 'MSD.py', 'moldyn/mace_finetuned_md.xyz', '--out', 'moldyn/msd_mace_finetuned.dat', '--dt', '1', '--png', 'moldyn/msd_mace_finetuned.png', '--skip', '1'], returncode=0)

In [16]:
# === Create temp and energy graphics for all MDs ===

mol_mace_data = np.load(os.path.join(output_dir, "mol_mace_4000.npz"))
mol_xtb_data = np.load(os.path.join(output_dir, "mol_xtb.npz"))

liq_mace_data = np.load(os.path.join(output_dir, "liq_mace_4000.npz"))
liq_ft_data = np.load(os.path.join(output_dir, "liq_finetuned.npz"))

# Load arrays
mol_mace_t, mol_mace_temp, mol_mace_E = mol_mace_data['t'], mol_mace_data['T'], mol_mace_data['E']
mol_xtb_t, mol_xtb_temp, mol_xtb_E = mol_xtb_data['t'], mol_xtb_data['T'], mol_xtb_data['E']
liq_mace_t, liq_mace_temp, liq_mace_E = liq_mace_data['t'], liq_mace_data['T'], liq_mace_data['E']
liq_ft_t, liq_ft_temp, liq_ft_E = liq_ft_data['t'], liq_ft_data['T'], liq_ft_data['E']

# Load constants from any of the files for the system
N_mol = int(mol_mace_data['N'])
T_target_mol = float(mol_mace_data['T_target'])

N_liq = int(liq_mace_data['N'])
T_target_liq = float(liq_mace_data['T_target'])



plot_temperature(mol_mace_t, mol_mace_temp, mol_xtb_t, mol_xtb_temp,
                 "MACE-4000", "XTB", temp_target=T_target_mol, N=N_mol,
                 out_file=os.path.join(output_dir, "temperature_molecule.png"))

plot_energy(mol_mace_t, mol_mace_E, mol_xtb_t, mol_xtb_E,
            "MACE-4000", "XTB",
            out_file=os.path.join(output_dir, "energy_molecule.png"))


plot_temperature(liq_mace_t, liq_mace_temp, liq_ft_t, liq_ft_temp,
                 "MACE-4000", "Fine-tuned", temp_target=T_target_liq, N=N_liq,
                 out_file=os.path.join(output_dir, "temperature_liquid.png"),
                 ylim=(200, 800))

plot_energy(liq_mace_t, liq_mace_E, liq_ft_t, liq_ft_E,
            "MACE-4000", "Fine-tuned",
            out_file=os.path.join(output_dir, "energy_liquid.png"))

In [18]:
# === Create rdf graphics for all MDs ===

rdf_tags = ['HO_inter', 'OO_inter', 'CC_inter'] 

# Molecule RDFs
#plot_rdfs(os.path.join(output_dir, "xtb_md_molecule.xyz"),
#          os.path.join(output_dir, "mace_md_molecule.xyz"),
#          tags=rdf_tags,
#          out_prefix=os.path.join(output_dir, "molecule"),
#          label1="XTB",
#          label2="MACE-4000")

# Liquid RDFs
plot_rdfs(os.path.join(output_dir, "mace_md_input2.xyz"),
          os.path.join(output_dir, "mace_finetuned_md.xyz"),
          tags=rdf_tags,
          out_prefix=os.path.join(output_dir, "liquid"),
          label1="MACE-4000",
          label2="Fine-tuned")


In [11]:
# === Create MSD graphics for all MDs ===

# Molecule MSD
t_mace, msd_mace = load_msd(os.path.join(output_dir, "msd_mace_md_molecule.dat"))
t_xtb, msd_xtb = load_msd(os.path.join(output_dir, "msd_xtb_md_molecule.dat"))

D_mace, _ = compute_diffusion(t_mace, msd_mace, t_max_fs=100)
D_xtb, _ = compute_diffusion(t_xtb, msd_xtb, t_max_fs=100)

plt.figure()
plt.plot(t_mace, msd_mace, label='MACE-4000')
plt.plot(t_xtb, msd_xtb, label='XTB')
plt.xlabel('Time [fs]')
plt.ylabel('MSD [Å²]')
plt.text(0.95, 0.05,
         f"D(MACE) = {D_mace:.4f} Å²/ps\nD(XTB) = {D_xtb:.4f} Å²/ps",
         transform=plt.gca().transAxes,
         ha='right', va='bottom',
         fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.6))
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "msd_molecule.png"), dpi=300)
plt.close()

# Liquid MSD
t_liq, msd_liq = load_msd(os.path.join(output_dir, "msd_mace_input2.dat"))
t_ft, msd_ft = load_msd(os.path.join(output_dir, "msd_mace_finetuned.dat"))

D_liq, _ = compute_diffusion(t_liq, msd_liq, t_max_fs=100)
D_ft, _ = compute_diffusion(t_ft, msd_ft, t_max_fs=100)

plt.figure()
plt.plot(t_liq, msd_liq, label='MACE-4000')
plt.plot(t_ft, msd_ft, label='Fine-tuned')
plt.xlabel('Time [fs]')
plt.ylabel('MSD [Å²]')
plt.text(0.95, 0.05,
         f"D(MACE) = {D_liq:.4f} Å²/ps\nD(Fine-tuned) = {D_ft:.4f} Å²/ps",
         transform=plt.gca().transAxes,
         ha='right', va='bottom',
         fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.6))
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "msd_liquid.png"), dpi=300)
plt.close()
